In [ ]:
import itertools
import subprocess
from collections import defaultdict
import math

In [ ]:
var_dict = {}
rev_dict = {}
last_id = 1

In [ ]:
def add_var_id(x, old_id=None):
    global last_id, var_dict, rev_dict
    if old_id != None:
        var_dict[x] = old_id
        return
    var_dict[x] = last_id
    x = x + '}'
    x = x[0] + '_{' + x[1:]
    rev_dict[str(last_id)] = x
    rev_dict['-' + str(last_id)] = x
    last_id += 1
    return str(last_id - 1)

In [ ]:
# k clause must be true
def generate_clauses(prefix, k, bool_vars):
    clauses = []
    bool_list = [x for x in bool_vars]
    for combination in itertools.combinations(bool_list, k + 1):
        clause = prefix + " ".join([f"-{x}" for x in combination])
        clauses.append(clause)
    
    for combination in itertools.combinations(bool_list, len(bool_list) - k + 1):
        clause = prefix + " ".join([f"{x}" for x in combination])
        clauses.append(clause)
    return clauses

In [ ]:
def srg(n, la, mu):
    angle = [[set() for j in range(n)] for i in range(n)]
    clauses = []
    
    for i, j in itertools.combinations(range(n), 2):
        edge_id = add_var_id(f"e{i},{j}")
        add_var_id(f"e{j},{i}", edge_id)
    
    # generate angle literals
    for i, j in itertools.combinations(range(n), 2):
        for k in range(n):
            if i == k or j == k:
                continue
            angle_id = add_var_id(f"t{i},{j},{k}")
            angle[i][j].add(angle_id)
            edge_1 = var_dict[f"e{i},{k}"]
            edge_2 = var_dict[f"e{j},{k}"]
            clauses.append(f"{angle_id} -{edge_1} -{edge_2}")
            clauses.append(f"-{angle_id} {edge_1}")
            clauses.append(f"-{angle_id} {edge_2}")
    
    for i in range(n):
        for j in range(i + 1, n):
            edge = var_dict[f"e{i},{j}"]
            clauses.extend(generate_clauses(f"-{edge} ", la, angle[i][j]))
    
    for i in range(n):
        for j in range(i + 1, n):
            edge = var_dict[f"e{i},{j}"]
            clauses.extend(generate_clauses(f"{edge} ", mu, angle[i][j]))
    for clause in clauses:
        ans = ''.join(clause)
    return clauses

In [ ]:
def clause_to_cnf(clauses):
    cnf_file = "p cnf {} {}\n".format(last_id - 1,len(clauses))

    for s in clauses:
        cnf_file = cnf_file + s + ' 0\n'
    with open(f'srgs_cnfs/srgN{N}L{LAMBDA}M{MU}.cnf', 'w') as f:
        f.write(cnf_file)
    return cnf_file

In [ ]:
def clause_count(N, MU, LAMBDA):
    term1 = math.comb(N - 2, MU - 1)
    term2 = math.comb(N - 2, MU + 1)
    term3 = math.comb(N - 2, LAMBDA + 1)
    term4 = math.comb(N - 2, LAMBDA - 1)

    result = term1 + term2 + term3 + term4
    result *= math.comb(N, 2)

    return result

In [ ]:
N, LAMBDA, MU = 9, 1, 2
print(clause_count(N, MU, LAMBDA)/1e8)

In [ ]:
c = srg(N, LAMBDA, MU)
cnf_file = clause_to_cnf(c)

In [ ]:
# Run CaDiCaL with the input file and capture the output and any errors
result = subprocess.run(['/usr/local/bin/cadical'], input=cnf_file.encode('utf-8'), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
edges_list = []
output = result.stdout.decode('utf-8')
v_lines = [line for line in output.split('\n') if line.startswith('v')]
ans = '\n'.join(v_lines)
ans = ans.replace('v','')
ans = ans.replace('\n','')
numbers_list = [int(num) for num in ans.split()]
for num in numbers_list:
    if num > 0:
        res = rev_dict[str(abs(num))]
        if res[0] == 'e':
            edges_list.append(parse_edge_string(res))

In [ ]:
def check_srg(N, LAMBDA, MU, edges_list):
    adjacency_list = defaultdict(set)
    degrees = defaultdict(int)
    for u, v in edges_list:
        adjacency_list[u].add(v)
        adjacency_list[v].add(u)
        degrees[u] += 1
        degrees[v] += 1

    k = degrees[1]
    print("Regularity found is", k)
    if not all(degree == k for degree in degrees.values()):
        return False

    for u, v in edges_list:
        common_neighbors = adjacency_list[u] & adjacency_list[v]
        if len(common_neighbors) != LAMBDA:
            return False

    for u in range(N):
        for v in range(u + 1, N):
            if (u, v) not in edges_list and (v, u) not in edges_list:
                common_neighbors = adjacency_list[u] & adjacency_list[v]
                if len(common_neighbors) != MU:
                    return False

    return True

In [ ]:
def graph_to_csv(n, edges):
    header = ";" + ";".join(str(i) for i in range(n)) + "\n"
    
    row_strings = []
    for i in range(n):
        row = [str(i)] + ["0" if (i, j) not in edges and (j, i) not in edges else "1" for j in range(n)]
        row_strings.append(";".join(row) + "\n")
    csv_content = header + "".join(row_strings)
    
    with open(f"srgs_csvs/srgN{N}L{LAMBDA}M{MU}.csv", "w") as file:
        file.write(csv_content)

if check_srg(N, LAMBDA, MU, edges_list):
    print("Result is strongly regular")
elif len(edges_list) == 0:
    print("UNSATISFIABLE")
else:
    print("Wrong result has been generated.")
graph_to_csv(N, edges_list)